In [8]:
import librosa
import numpy as np


def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None, mono=True)
    duration = librosa.get_duration(y=y, sr=sr)
    
    # Beat Tracking
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
    beat_density = len(beats) / duration if duration > 0 else 0
    
    # Spectral Features
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    
    # Pitch Features
    pitch, voiced_flag, voiced_prob = librosa.pyin(
        y,
        fmin=float(librosa.note_to_hz('C2')),
        fmax=float(librosa.note_to_hz('C7'))
    )
    pitch = pitch[~np.isnan(pitch)]
    pitch_range = np.max(pitch) - np.min(pitch) if len(pitch) > 0 else 0
    
    rms = librosa.feature.rms(y=y)
    dynamic_range = np.max(rms) - np.min(rms)
    
    return {
        "tempo": tempo,
        "beat_density": beat_density,
        "spectral_centroid": spectral_centroid,
        "spectral_bandwidth": spectral_bandwidth,
        "spectral_rolloff": spectral_rolloff,
        "pitch_range": pitch_range,
        "dynamic_range": dynamic_range
    }

In [9]:
import os


def process_artist_folder(folder_path):
    song_features = []

    for file in os.listdir(folder_path):
        if file.lower().endswith(".mp3"):
            file_path = os.path.join(folder_path, file)
            features = extract_features(file_path)
            song_features.append(features)

    return song_features

In [10]:
def aggregate_artist_style(song_features):
    tempos = [s["tempo"] for s in song_features]
    pitch_ranges = [s["pitch_range"] for s in song_features]
    centroids = [s["spectral_centroid"] for s in song_features]
    dynamics = [s["dynamic_range"] for s in song_features]

    return {
        "tempo_range": f"{int(np.percentile(tempos, 25))}–{int(np.percentile(tempos, 75))} BPM",
        "avg_tempo": np.mean(tempos),
        "melodic_range": "narrow" if np.mean(pitch_ranges) < 800 else "wide",
        "brightness": "dark" if np.mean(centroids) < 2500 else "bright",
        "dynamic_profile": "compressed" if np.mean(dynamics) < 0.05 else "dynamic"
    }


In [11]:
folder = "ip_audio/sickick"
song_data = process_artist_folder(folder)
artist_style = aggregate_artist_style(song_data)

print(artist_style)


{'tempo_range': '126–129 BPM', 'avg_tempo': np.float64(123.92352475128371), 'melodic_range': 'narrow', 'brightness': 'bright', 'dynamic_profile': 'dynamic'}
